In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import time
import mysql.connector
from datetime import datetime, timedelta
con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345678",
    database="REDBUS"
)
cursor = con.cursor()
cursor
driver=webdriver.Chrome()

state_transports=["https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",      
                  "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/north-bengal-state-transport-corporation",
                  "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu",
                  "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"]
for transport in state_transports:
    driver.get(transport)
    driver.maximize_window()
    time.sleep(3)
    route_names=[]
    route_links=[]
    #pagelink
    pages=driver.find_element(By.CLASS_NAME,"DC_117_paginationTable")
    page_tabs=driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ")
    for i in range(1,len(page_tabs)+1):
        if i>1:
            next_page=pages.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{i}"]')
            actions = ActionChains(driver)
            actions.move_to_element(next_page).perform()
            time.sleep(2)
            next_page.click()  
        routes=driver.find_elements(By.CLASS_NAME, "route")
        for route in routes:
            route_names.append(route.text)
        links=[route.get_attribute('href') for route in routes]
        for link in links:
            route_links.append(link)
    
    for link,route in zip(route_links,route_names):
        driver.get(link)
        #driver.maximize_window()
        time.sleep(3)
    
        try:
            viewbutton = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='button']"))
            )
            viewbutton.click()
            time.sleep(3)
        except Exception as e:
            print(f"Error clicking view button on {link}: {e}")
    
        
        #viewbutton=driver.find_element(By.XPATH,"//div[@class='button']")
        #viewbutton.click()
        #time.sleep(3)
        
        counter=0
        max_scroll=15
        while counter < max_scroll:
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(5)
            counter+=1
        bus_names=[]
        bus_types=[]
        departing_times=[]
        durations=[]
        reaching_times=[]
        star_ratings=[]
        prices=[]
        seat_availabilities=[]    
        bus_name=driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
        bus_type=driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time=driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
        duration=driver.find_elements(By.XPATH,"//div[@class='column-four p-right-10 w-10 fl']")
        reaching_time=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating=driver.find_elements(By.XPATH,"//div[@class='rating-sec lh-24']")
        price=driver.find_elements(By.XPATH,"//span[@class='f-19 f-bold']")
        seat_availability=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
        for name in bus_name:
            bus_names.append(name.text)    
        for typ in bus_type:
            bus_types.append(typ.text)
        for departing in departing_time:
            departing_times.append(departing.text)
        for dura in duration:
            durations.append(dura.text)
        for reaching in reaching_time:
            reaching_times.append(reaching.text) 
        for star in star_rating:
            star_ratings.append(star.text) 
        for pri in price:
            prices.append(pri.text) 
        for seat in seat_availability:
            seat_availabilities.append(seat.text.split()[0]) 
        for(name,typ,departing,dur,reaching,star,pri,seat) in zip(bus_names,bus_types,departing_times,durations,reaching_times,star_ratings,prices,seat_availabilities):
            query="insert into bus_route (routename,routelink,busname,bustype,departingtime,duration,reachingtime,starrating,price,seatavailable) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values=(route,link,name,typ,departing,dur,reaching,star,pri,seat)
            cursor.execute(query,values)
            con.commit()

Error clicking view button on https://www.redbus.in/bus-tickets/bangalore-to-kozhikode: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF753B8EEB2+31554]
	(No symbol) [0x00007FF753B07EE9]
	(No symbol) [0x00007FF7539C872A]
	(No symbol) [0x00007FF75399D995]
	(No symbol) [0x00007FF753A444D7]
	(No symbol) [0x00007FF753A5C051]
	(No symbol) [0x00007FF753A3CDD3]
	(No symbol) [0x00007FF753A0A33B]
	(No symbol) [0x00007FF753A0AED1]
	GetHandleVerifier [0x00007FF753E98B2D+3217341]
	GetHandleVerifier [0x00007FF753EE5AF3+3532675]
	GetHandleVerifier [0x00007FF753EDB0F0+3489152]
	GetHandleVerifier [0x00007FF753C3E786+750614]
	(No symbol) [0x00007FF753B1376F]
	(No symbol) [0x00007FF753B0EB24]
	(No symbol) [0x00007FF753B0ECB2]
	(No symbol) [0x00007FF753AFE17F]
	BaseThreadInitThunk [0x00007FF9F0367374+20]
	RtlUserThreadStart [0x00007FF9F06DCC91+33]



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF753B8EEB2+31554]
	(No symbol) [0x00007FF753B07EE9]
	(No symbol) [0x00007FF7539C872A]
	(No symbol) [0x00007FF75399D995]
	(No symbol) [0x00007FF753A444D7]
	(No symbol) [0x00007FF753A5C051]
	(No symbol) [0x00007FF753A3CDD3]
	(No symbol) [0x00007FF753A0A33B]
	(No symbol) [0x00007FF753A0AED1]
	GetHandleVerifier [0x00007FF753E98B2D+3217341]
	GetHandleVerifier [0x00007FF753EE5AF3+3532675]
	GetHandleVerifier [0x00007FF753EDB0F0+3489152]
	GetHandleVerifier [0x00007FF753C3E786+750614]
	(No symbol) [0x00007FF753B1376F]
	(No symbol) [0x00007FF753B0EB24]
	(No symbol) [0x00007FF753B0ECB2]
	(No symbol) [0x00007FF753AFE17F]
	BaseThreadInitThunk [0x00007FF9F0367374+20]
	RtlUserThreadStart [0x00007FF9F06DCC91+33]
